# PrepareData2 - Q_D

Prepares data for script which are:

- Target: difference in quantile of scores

In [1]:
import pandas as pd

from sklearn.model_selection import KFold
from sklearn import preprocessing

import numpy as np

import os

In [2]:
if not os.path.exists('./Q_D_PreparedData'):
    os.mkdir('./Q_D_PreparedData')

In [3]:
kf = KFold(n_splits = 5, shuffle = True, random_state=42)

In [4]:
def manipdata(newStdData, NWEEKS, attrb, meth, maniptype, discard):
    
    metadata = ['ID', 'Domain', 'Week', 'Selfweeks']
    masterlist = [list() for i in range(NWEEKS+5)]
    
    skip = 0
    if discard:
        skip = 26
    
    
    for i in range(len(newStdData)):
        if (newStdData.loc[i]['Selfweeks'] > NWEEKS+skip) and (i+1 < len(newStdData)) and (newStdData.loc[i]['Domain'] == newStdData.loc[i+1]['Domain']):
            
            for j in range(len(metadata)):
                masterlist[j].append(newStdData.loc[i][metadata[j]])
                
            for j in range(4, NWEEKS+4):
                masterlist[j].append(newStdData.loc[i-(j-4)][attrb])
            
            masterlist[NWEEKS+4].append(newStdData.loc[i][f'{attrb}T'])

    out = pd.DataFrame()
        
    for i in range(NWEEKS+5):
        if i < 4:
            out.insert(i, metadata[i], masterlist[i])
        elif i == NWEEKS + 4:
            out.insert(i, 'Target', masterlist[i])
        else:
            out.insert(i, f'{i-4}', masterlist[i])
            
    out.to_csv(f'./Q_D_PreparedData/{maniptype}_{meth}_{attrb}_{NWEEKS}.csv', index = False)


In [ ]:
for meth in ['A', '-26']:
    df = pd.read_csv('./Partial_Output/_3_combined_cleaned_data.csv')
    df = df[df.columns[:-1]]
    
    discard = False
    
    if '-26' in meth:
        df = df[df['Selfweeks'] > 26]
        df.index = range(len(df))
        
        discard = True
    
    data = pd.DataFrame()
    for domain, tmp in df.groupby('Domain'):
        tmp.index = range(len(tmp))
        
        for i in range(len(tmp)-1, -1, -1):
            if ((tmp.loc[i]['Assets']) | (tmp.loc[i]['Actions']) | (tmp.loc[i]['Competency']) |
                    (tmp.loc[i]['Form_record']) | (tmp.loc[i]['Form_template']) |
                    (tmp.loc[i]['Incident']) | (tmp.loc[i]['Users'])):
                break
        
        data = pd.concat([data, tmp[0:i+1]])
        
    stdData = pd.DataFrame()
    
    for domain, compData in data.groupby('Domain'):
    
        compDataX = compData[['Assets', 'Actions', 'Competency',
           'Form_record', 'Form_template', 'Incident', 'Users']]

        scaler = preprocessing.StandardScaler().fit(compDataX)

        compData[['Assets', 'Actions', 'Competency',
           'Form_record', 'Form_template', 'Incident', 'Users']] = scaler.transform(compDataX)

        stdData = pd.concat([stdData, compData])
    
    
    newStdData = pd.DataFrame()
    for colName in stdData.columns[4:]:

        target = list()

        for domain, compData in stdData.groupby('Domain'):
            
            compData.index = compData['Selfweeks']
            
            index = compData.index
            
            out = [(compData.loc[i+1][colName]-compData.loc[i][colName]) if (i+1 in index)
                   else np.nan for i in index]
            
            target.extend(out)

        stdData[f'{colName}T'] = target
        
    for domain, data in stdData.groupby('Week'): #注意这里只是给data加了一个column！
        for colName in stdData.columns[4:13]:
            dmin = data[f'{colName}T'].min()
            dmax = data[f'{colName}T'].max()
            data[f'{colName}T'] = (data[f'{colName}T']-dmin)/(dmax-dmin) #问题
        newStdData = pd.concat([newStdData, data])
            
    newStdData['Week'].astype(int)
    newStdData = newStdData.sort_values(['Domain', 'Week'])
    newStdData.index = range(len(newStdData))
    
    for attrb in newStdData.columns[4:11]:
        for nWeeks in range(1, 13):
            manipdata(newStdData, nWeeks, attrb, meth, 'Q_D', discard)
            

In [ ]:
if not os.path.exists('./Q_D_SplitData'):
    os.mkdir('./Q_D_SplitData')

In [ ]:
filelist = os.listdir("./Q_D_PreparedData")
for file in filelist:
    if file[0] != '.':
        stdData = pd.read_csv(f"./Q_D_PreparedData/{file}")
        fold = 1
        
        startweek = int(file.strip('.csv').split('_')[-1])
        
        if '-26' in file:
            startweek += 26
        
        for trainWeeks, testWeeks in kf.split(range(startweek, 215)):

            testBool = stdData.Week.isin(testWeeks)
            testData = stdData[testBool]

            trainBool = stdData.Week.isin(trainWeeks)
            trainData = stdData[trainBool]

            testData.to_csv(f'./Q_D_SplitData/{file.strip(".csv")}_Test_{fold}.csv', index = False)
            trainData.to_csv(f'./Q_D_SplitData/{file.strip(".csv")}_Train_{fold}.csv', index = False)

            fold += 1
    